5)This script is 'final_code' with non-binary games which has one more feature(value of each cell 0,1,2) compared to 'final_code'.the row and column of each element, the distance to the edges, proximity to the red or blue wall, and the value of each node.

In [190]:
from GraphTsetlinMachine.graphs import Graphs
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time
import argparse
from skimage.util import view_as_windows
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib

In [209]:
import pandas as pd
import networkx as nx
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib

# Load the CSV file generated by C program
df = pd.read_csv('hex_game_results_13.csv')

# Limit to 10,000 rows(just for more speed processing)
df = df.iloc[:10000]

# Convert 'Moves' column to lists of integers
df['Moves'] = df['Moves'].apply(lambda x: list(map(int, x.split())))

# Check the processed moves
print("Processed Moves:")
print(df['Moves'].head())

# Assuming board size of 13x13 for the Hex game
BOARD_SIZE = 13

# Function to convert moves into a final board state
def get_final_board_state(moves, board_size):
    board = np.zeros((board_size, board_size), dtype=int)
    for i, move in enumerate(moves):
        player = 1 if i % 2 == 0 else 2
        row, col = move // board_size, move % board_size
        board[row, col] = player
    return board

# Add the final board state to the DataFrame
df['FinalBoard'] = df['Moves'].apply(lambda moves: get_final_board_state(moves, BOARD_SIZE))

# Check the final board states
print("Final Board State for the first game:")
print(df['FinalBoard'].iloc[0])

# Split the data into training and testing sets
# We have a 'Winner' column in the dataset for labels
X_train, X_test, Y_train, Y_test = train_test_split(df['FinalBoard'], df['Winner'], test_size=0.2, random_state=42)

# Check the split data
print(f"Training set size: {len(X_train)}")
print(f"Testing set size: {len(X_test)}")

X_train.iloc[0]

Processed Moves:
0    [137, 54, 139, 138, 73, 75, 148, 130, 13, 135,...
1    [49, 29, 81, 0, 115, 104, 41, 40, 105, 76, 144...
2    [142, 139, 97, 56, 33, 110, 35, 167, 76, 48, 1...
3    [13, 158, 102, 49, 101, 54, 18, 71, 108, 26, 4...
4    [3, 51, 103, 95, 11, 165, 98, 47, 49, 46, 55, ...
Name: Moves, dtype: object
Final Board State for the first game:
[[2 1 2 1 1 0 1 2 1 1 2 2 2]
 [1 1 1 2 2 0 1 2 1 1 1 0 2]
 [2 2 2 1 2 2 1 2 0 1 1 0 1]
 [2 2 0 1 1 2 1 2 1 2 1 2 1]
 [1 0 2 1 2 2 0 2 1 2 2 1 2]
 [0 0 2 2 1 1 1 0 1 2 2 2 2]
 [1 1 1 2 2 2 2 1 1 2 1 0 1]
 [1 2 1 2 0 0 2 0 1 1 1 2 2]
 [2 2 1 0 2 2 0 0 1 2 1 2 2]
 [2 2 0 1 1 1 1 2 1 0 1 0 1]
 [2 2 0 1 1 2 2 1 2 1 1 2 1]
 [1 0 2 1 2 1 2 1 1 2 0 2 1]
 [1 1 2 1 2 1 2 2 1 0 1 0 2]]
Training set size: 8000
Testing set size: 2000


array([[1, 0, 0, 2, 2, 1, 1, 1, 2, 1, 2, 0, 1],
       [2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2],
       [2, 0, 2, 2, 1, 2, 2, 2, 1, 2, 1, 0, 2],
       [1, 1, 2, 1, 2, 1, 0, 1, 0, 1, 2, 2, 1],
       [1, 2, 0, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1],
       [1, 1, 2, 0, 2, 2, 1, 1, 1, 0, 1, 2, 2],
       [0, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2],
       [2, 2, 2, 1, 2, 1, 2, 0, 1, 1, 2, 2, 2],
       [1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 2],
       [1, 1, 2, 1, 1, 2, 1, 0, 2, 1, 1, 1, 2],
       [1, 1, 1, 0, 1, 2, 2, 1, 0, 2, 2, 0, 2],
       [2, 2, 2, 1, 2, 2, 1, 2, 0, 1, 2, 2, 0],
       [1, 1, 1, 2, 1, 0, 1, 2, 0, 1, 2, 2, 2]])

In [210]:
Y_train[:10]

9254    1
1561    1
1670    2
6087    2
6669    1
5933    2
8829    2
7945    1
3508    1
2002    1
Name: Winner, dtype: int64

In [211]:
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
X_train_list = np.array(X_train_list)
X_test_list = np.array(X_test_list)
print(len(X_train_list))
X_train_list.shape

8000


(8000, 13, 13)

In [212]:
X_train_list[0,0,0]

1

In [213]:
from GraphTsetlinMachine.graphs import Graphs
import numpy as np
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
from time import time


dim = 13

number_of_nodes = dim * dim


symbol_names = ['0','1','2']

# Distance to walls (for a 13x13 grid, distances from 0 to 6)
for dist in range(dim // 2 + 1):
    symbol_names.append(f'DistToWall_{dist}')

# Adjacency to walls
symbol_names.extend(['TouchingRedWall', 'TouchingBlueWall'])

# Row and column information
for row in range(dim):
    symbol_names.append(f'Row_{row}')
for col in range(dim):
    symbol_names.append(f'Col_{col}')

symbol_names

['0',
 '1',
 '2',
 'DistToWall_0',
 'DistToWall_1',
 'DistToWall_2',
 'DistToWall_3',
 'DistToWall_4',
 'DistToWall_5',
 'DistToWall_6',
 'TouchingRedWall',
 'TouchingBlueWall',
 'Row_0',
 'Row_1',
 'Row_2',
 'Row_3',
 'Row_4',
 'Row_5',
 'Row_6',
 'Row_7',
 'Row_8',
 'Row_9',
 'Row_10',
 'Row_11',
 'Row_12',
 'Col_0',
 'Col_1',
 'Col_2',
 'Col_3',
 'Col_4',
 'Col_5',
 'Col_6',
 'Col_7',
 'Col_8',
 'Col_9',
 'Col_10',
 'Col_11',
 'Col_12']

In [214]:
# Initialize the graph object with appropriate symbols and settings
graphs_train = Graphs(X_train_list.shape[0], symbols=symbol_names, hypervector_size=128, hypervector_bits=2)

# Step 2: Set the number of graph nodes for each training graph
for graph_id in range(X_train_list.shape[0]):
    graphs_train.set_number_of_graph_nodes(graph_id, dim * dim)

# Step 3: Prepare the node configuration
graphs_train.prepare_node_configuration()

# Step 4: Add nodes for each graph
for graph_id in range(X_train_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col  # Unique node ID for each hexagon

            # Dynamically calculate the number of neighbors for each node
            number_of_neighbors = 0
            if row > 0: number_of_neighbors += 1  # Above
            if row < dim - 1: number_of_neighbors += 1  # Below
            if col > 0: number_of_neighbors += 1  # Left
            if col < dim - 1: number_of_neighbors += 1  # Right
            if row > 0 and col < dim - 1: number_of_neighbors += 1  # AboveRight
            if row < dim - 1 and col > 0: number_of_neighbors += 1  # BelowLeft

            # Check for Node 0 (top-left corner)
            #if node_id == 0:
                #print(f"Node 0 (graph {graph_id}): number_of_neighbors = {number_of_neighbors}")
                # Ensure Node 0 has exactly 2 neighbors
                #assert number_of_neighbors == 2, f"Error: Node 0 has {number_of_neighbors} neighbors, expected 2."

            # define the nodes with the correct number of neighbors(dynamically)
            graphs_train.add_graph_node(graph_id, node_id, number_of_neighbors)

# Step 5: Add node properties for each graph
for graph_id in range(X_train_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col  # Unique node ID for each hexagon

            # Feature 1: Value of each node (convert to string)
            node_value = str(X_train_list[graph_id, row, col])
            if node_value in symbol_names:
                graphs_train.add_graph_node_property(graph_id, node_id, node_value)
            else:
                print(f"Warning: Node value {node_value} not found in symbol_names")

            # Feature 2: Distance to walls or center
            distance_to_wall = min(row, col, dim - row - 1, dim - col - 1)
            graphs_train.add_graph_node_property(graph_id, node_id, f'DistToWall_{distance_to_wall}')

            # Feature 3: Adjacency to walls
            if row == 0 or row == dim - 1:
                graphs_train.add_graph_node_property(graph_id, node_id, "TouchingRedWall")
            if col == 0 or col == dim - 1:
                graphs_train.add_graph_node_property(graph_id, node_id, "TouchingBlueWall")

            # Feature 4: Row and column information
            graphs_train.add_graph_node_property(graph_id, node_id, f'Row_{row}')
            graphs_train.add_graph_node_property(graph_id, node_id, f'Col_{col}')
# Step 6: Prepare the edge configuration for the training data
graphs_train.prepare_edge_configuration()

# Step 7: Add edges between nodes (neighboring hexagons) for the training data
for graph_id in range(X_train_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col

            # Handle Node 0 (top-left corner) separately to avoid adding too many edges
            if node_id == 0:
                # Node 0 can only connect to the right and below
                #print(f"Adding edges for Node 0 in graph {graph_id}")
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id + dim, "Below")  # Connect to node below
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id + 1, "Right")    # Connect to node on the right
                #print(f"Node 0 edges: Below -> {node_id + dim}, Right -> {node_id + 1}")
                continue  # Skip the rest of the logic for Node 0 to avoid adding more edges


            # Add edges to neighboring hexagons (hex grid has up to 6 neighbors per cell)
            if row > 0:  # Add edge to the hexagon above
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id - dim, "Above")
            if row < dim - 1:  # Add edge to the hexagon below
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id + dim, "Below")
            if col > 0:  # Add edge to the hexagon on the left
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id - 1, "Left")
            if col < dim - 1:  # Add edge to the hexagon on the right
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id + 1, "Right")

            # Optionally add diagonal neighbors (depending on your hex grid layout)
            if row > 0 and col < dim - 1:
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id - dim + 1, "AboveRight")
            if row < dim - 1 and col > 0:
                graphs_train.add_graph_node_edge(graph_id, node_id, node_id + dim - 1, "BelowLeft")

In [215]:
# Check if node still has no edges (for debugging)
if not graphs_train.graph_node_id[graph_id][node_id]:
                print(f"Warning: Node {node_id} in graph {graph_id} has no edges added.")

In [216]:
# Step 8: After adding nodes and edges, encode the graph for the training data
graphs_train.encode()

print("Training data with new features produced")
print('finish_v')

Training data with new features produced
finish_v


In [217]:
# Initialize the graph object with appropriate symbols and settings
graphs_test = Graphs(X_test_list.shape[0], symbols=symbol_names, hypervector_size=128, hypervector_bits=2)

# Step 2: Set the number of graph nodes for each training graph
for graph_id in range(X_test_list.shape[0]):
    graphs_test.set_number_of_graph_nodes(graph_id, dim * dim)

# Step 3: Prepare the node configuration
graphs_test.prepare_node_configuration()

# Step 4: Add nodes for each graph
for graph_id in range(X_test_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col  # Unique node ID for each hexagon

            # Dynamically calculate the number of neighbors for each node
            number_of_neighbors = 0
            if row > 0: number_of_neighbors += 1  # Above
            if row < dim - 1: number_of_neighbors += 1  # Below
            if col > 0: number_of_neighbors += 1  # Left
            if col < dim - 1: number_of_neighbors += 1  # Right
            if row > 0 and col < dim - 1: number_of_neighbors += 1  # AboveRight
            if row < dim - 1 and col > 0: number_of_neighbors += 1  # BelowLeft

            # Check for Node 0 (top-left corner)
            #if node_id == 0:
                #print(f"Node 0 (graph {graph_id}): number_of_neighbors = {number_of_neighbors}")
                # Ensure Node 0 has exactly 2 neighbors
                #assert number_of_neighbors == 2, f"Error: Node 0 has {number_of_neighbors} neighbors, expected 2."

            # define the nodes with the correct number of neighbors(dynamically)
            graphs_test.add_graph_node(graph_id, node_id, number_of_neighbors)

# Step 5: Add node properties for each graph
for graph_id in range(X_test_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col  # Unique node ID for each hexagon

            # Feature 1: Value of each node (convert to string)
            node_value = str(X_test_list[graph_id, row, col])
            if node_value in symbol_names:
                graphs_test.add_graph_node_property(graph_id, node_id, node_value)
            else:
                print(f"Warning: Node value {node_value} not found in symbol_names")

            # Feature 2: Distance to walls or center
            distance_to_wall = min(row, col, dim - row - 1, dim - col - 1)
            graphs_test.add_graph_node_property(graph_id, node_id, f'DistToWall_{distance_to_wall}')

            # Feature 3: Adjacency to walls
            if row == 0 or row == dim - 1:
                graphs_test.add_graph_node_property(graph_id, node_id, "TouchingRedWall")
            if col == 0 or col == dim - 1:
                graphs_test.add_graph_node_property(graph_id, node_id, "TouchingBlueWall")

            # Feature 4: Row and column information
            graphs_test.add_graph_node_property(graph_id, node_id, f'Row_{row}')
            graphs_test.add_graph_node_property(graph_id, node_id, f'Col_{col}')
# Step 6: Prepare the edge configuration for the training data
graphs_test.prepare_edge_configuration()

# Step 7: Add edges between nodes (neighboring hexagons) for the training data
for graph_id in range(X_test_list.shape[0]):
    for row in range(dim):
        for col in range(dim):
            node_id = row * dim + col

            # Handle Node 0 (top-left corner) separately to avoid adding too many edges
            if node_id == 0:
                # Node 0 can only connect to the right and below
                #print(f"Adding edges for Node 0 in graph {graph_id}")
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id + dim, "Below")  # Connect to node below
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id + 1, "Right")    # Connect to node on the right
                #print(f"Node 0 edges: Below -> {node_id + dim}, Right -> {node_id + 1}")
                continue  # Skip the rest of the logic for Node 0 to avoid adding more edges


            # Add edges to neighboring hexagons (hex grid has up to 6 neighbors per cell)
            if row > 0:  # Add edge to the hexagon above
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id - dim, "Above")
            if row < dim - 1:  # Add edge to the hexagon below
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id + dim, "Below")
            if col > 0:  # Add edge to the hexagon on the left
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id - 1, "Left")
            if col < dim - 1:  # Add edge to the hexagon on the right
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id + 1, "Right")

            # Optionally add diagonal neighbors (depending on your hex grid layout)
            if row > 0 and col < dim - 1:
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id - dim + 1, "AboveRight")
            if row < dim - 1 and col > 0:
                graphs_test.add_graph_node_edge(graph_id, node_id, node_id + dim - 1, "BelowLeft")

# Step 8: After adding nodes and edges, encode the graph for the training data
graphs_test.encode()

print("Test data with new features produced_v")

Test data with new features produced_v


In [218]:
# Create the Graph Tsetlin Machine model
tm = MultiClassGraphTsetlinMachine(
    number_of_clauses=2000,  # Adjust based on complexity of the game
    T=2000,
    s=10.0,
    depth=1,
    message_size=256,
    message_bits=2,
    max_included_literals=64
)

# Train the model on the Hex game data
for i in range(70):  # Number of epochs
    start_training = time()
    tm.fit(graphs_train, Y_train, epochs=1, incremental=True)
    stop_training = time()

    print(f"Epoch {i + 1} completed in {stop_training - start_training:.2f} seconds")

# Evaluate the model
# Assuming you have similarly prepared graphs_test for test data
start_testing = time()
result_test = 100 * (tm.predict(graphs_test) ==Y_test).mean()
stop_testing = time()

print(f"Test accuracy: {result_test:.2f}%")

Initialization of sparse structure.
Epoch 1 completed in 2.08 seconds
Epoch 2 completed in 1.47 seconds
Epoch 3 completed in 1.46 seconds
Epoch 4 completed in 1.46 seconds
Epoch 5 completed in 1.47 seconds
Epoch 6 completed in 1.46 seconds
Epoch 7 completed in 1.47 seconds
Epoch 8 completed in 1.47 seconds
Epoch 9 completed in 1.46 seconds
Epoch 10 completed in 1.46 seconds
Epoch 11 completed in 1.46 seconds
Epoch 12 completed in 1.45 seconds
Epoch 13 completed in 1.45 seconds
Epoch 14 completed in 1.46 seconds
Epoch 15 completed in 1.45 seconds
Epoch 16 completed in 1.45 seconds
Epoch 17 completed in 1.46 seconds
Epoch 18 completed in 1.46 seconds
Epoch 19 completed in 1.45 seconds
Epoch 20 completed in 1.45 seconds
Epoch 21 completed in 1.44 seconds
Epoch 22 completed in 1.45 seconds
Epoch 23 completed in 1.46 seconds
Epoch 24 completed in 1.44 seconds
Epoch 25 completed in 1.44 seconds
Epoch 26 completed in 1.44 seconds
Epoch 27 completed in 1.45 seconds
Epoch 28 completed in 1.44 s

In [219]:
print("First 10 actual winners in Y_test:")
print(Y_test[:10])

First 10 actual winners in Y_test:
6252    1
4684    2
1731    1
4742    1
4521    2
6340    1
576     2
5202    2
6363    1
439     1
Name: Winner, dtype: int64


In [220]:
# Step 1: Predict the winners using the trained model
Y_pred = tm.predict(graphs_test)

# Step 4: View a subset of Y_pred (for example, first 10 predictions)
print("First 10 predicted winners in Y_pred:")
print(Y_pred[:10])

First 10 predicted winners in Y_pred:
[2 2 2 2 2 2 2 2 2 2]


In [221]:
min(Y_pred)

2